In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.ensemble import IsolationForest

from scipy import stats

# Load Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Wash-Trading-Case-Study-for-ERC20-Token-Group-1/data/
data=pd.read_excel("NEAR_token_transaction_data.xlsx")

MessageError: Error: credential propagation was unsuccessful

# Data Exploration


Data Exploration

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
data.columns

In [ ]:
data.dtypes

In [ ]:
data.describe()

In [ ]:
missing_values = data.isnull().sum()
missing_values

In [ ]:
data['Quantity'].unique()

In [ ]:
data['Quantity'].value_counts()

In [ ]:
data['Method'].unique()

In [ ]:
data['Method'].value_counts()

# Exploratory Data Analysis


In [ ]:
data['Quantity'] = pd.to_numeric(data['Quantity'], errors='coerce')

In [ ]:
sample_data = data.sample(n=1000, random_state=42)

In [ ]:
plt.figure(figsize=(10, 6))

plt.title('Histogram of Transaction Quantities', fontsize=14)
plt.xlabel('Quantity', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
counts, bins, bars = plt.hist(sample_data['Quantity'], bins=50, color='skyblue', edgecolor='black')
for count, bar in zip(counts, bars):
    height = bar.get_height()
    if height > 0:
        plt.text(bar.get_x() + bar.get_width() / 2, height, int(count),
                 ha='center', va='bottom', fontsize=10)
plt.show()

In [ ]:
datetime = pd.to_datetime(data['DateTime (UTC)'])
quantity= pd.to_numeric(data['Quantity'], errors='coerce')

plt.figure(figsize=(12, 6))
plt.plot(datetime, quantity, marker='o', linestyle='-', color='b')
plt.title('Quantity Over Time')
plt.xlabel('DateTime')
plt.ylabel('Quantity')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
top_methods=data['Method'].value_counts().head(10)
fd=data[data['Method'].isin(top_methods.index)]
plt.figure(figsize=(10,6))
top_methods.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Top 10 Methods by Usage', fontsize=14)
plt.xlabel('Method', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
for i, count in enumerate(top_methods):
    plt.text(i, count, str(count), ha='center', va='bottom', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
top_methods = data['Method'].value_counts().head(10).index
filtered_data = data[data['Method'].isin(top_methods)]
quantity_per_method = filtered_data.groupby('Method')['Quantity'].sum()
plt.figure(figsize=(12, 6))
quantity_per_method.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Total Quantity per Method for Top 10 Methods (Sampled Data)', fontsize=14)
plt.xlabel('Method', fontsize=12)
plt.ylabel('Total Quantity', fontsize=12)
for i, quantity in enumerate(quantity_per_method):
    plt.text(i, quantity, f'{quantity:.2f}', ha='center', va='bottom', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
top_methods=data['From'].value_counts().head(10)
fd=data[data['From'].isin(top_methods.index)]
plt.figure(figsize=(10,6))
top_methods.plot(kind='bar', color='red', edgecolor='black')
plt.title('Top 10 Sellers', fontsize=14)
plt.xlabel('From', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
for i, count in enumerate(top_methods):
    plt.text(i, count, str(count), ha='center', va='bottom', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
top_methods=data['To'].value_counts().head(10)
fd=data[data['To'].isin(top_methods.index)]
plt.figure(figsize=(10,6))
top_methods.plot(kind='bar', color='red', edgecolor='black')
plt.title('Top 10 Buyers', fontsize=14)
plt.xlabel('To', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
for i, count in enumerate(top_methods):
    plt.text(i, count, str(count), ha='center', va='bottom', fontsize=10)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(data=data, x='Quantity', color='skyblue')
plt.title('Boxplot of Transaction Quantities (Outlier Detection)', fontsize=14)
plt.xlabel('Quantity', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## Network Analysis


In [ ]:
import networkx as nx

# Initialize a directed graph
G = nx.DiGraph()

# Add edges with weights (Quantity)
for _, row in data.iterrows():
    G.add_edge(row['From'], row['To'], weight=row['Quantity'])


In [ ]:
degree_centrality = nx.degree_centrality(G)


In [ ]:
betweenness_centrality = nx.betweenness_centrality(G, weight='weight')



In [ ]:
closeness_centrality = nx.closeness_centrality(G)


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

def plot_centrality(graph, centrality, title, top_n=None, show_labels=False, node_size_factor=5000):
    """
    Plots a centrality graph with optional focus on top N nodes and dynamic labeling.

    Arguments:
        graph: NetworkX graph.
        centrality: Dictionary of centrality scores.
        title: Title of the graph.
        top_n: Number of top nodes to display (optional).
        show_labels: Whether to display node labels (default: False).
        node_size_factor: Scaling factor for node sizes (default: 5000).
    """
    plt.figure(figsize=(12, 8))

    # Sort centrality by value (descending)
    sorted_centrality = sorted(centrality.items(), key=lambda x: x[1], reverse=True)

    # Focus on top N nodes (optional)
    if top_n:
        nodes_to_draw = {node for node, _ in sorted_centrality[:top_n]}
        graph = graph.subgraph(nodes_to_draw)
        centrality = {node: centrality[node] for node in nodes_to_draw}

    # Position nodes using spring layout
    pos = nx.spring_layout(graph, seed=42)
    centrality_values = list(centrality.values())

    # Normalize centrality values for color mapping
    vmin = min(centrality_values)
    vmax = max(centrality_values)

    # Draw nodes with size based on centrality and color
    nodes = nx.draw_networkx_nodes(
        graph, pos,
        node_color=centrality_values,
        cmap=plt.cm.viridis,
        node_size=[node_size_factor * val for val in centrality_values],
        vmin=vmin,
        vmax=vmax
    )
    nx.draw_networkx_edges(graph, pos, alpha=0.3, edge_color="gray")

    # Add labels for top N nodes if show_labels is True
    if show_labels:
        nx.draw_networkx_labels(
            graph, pos,
            labels={node: node for node in centrality.keys()},
            font_size=8, font_color="black"
        )

    # Add colorbar explicitly linked to the nodes
    cbar = plt.colorbar(nodes, shrink=0.8)
    cbar.set_label("Centrality Score")

    # Add title
    plt.title(title, fontsize=14)
    plt.axis("off")  # Turn off axis
    plt.show()

In [ ]:
plot_centrality(G, degree_centrality, 'Degree Centrality', top_n=10, show_labels=True)


In [ ]:
plot_centrality(G, betweenness_centrality, 'Betweenness Centrality', top_n=10, show_labels=True)

In [ ]:
plot_centrality(G, closeness_centrality, 'Closeness Centrality', top_n=10, show_labels=True)

# Data Cleaning

In [ ]:
missing_data = pd.DataFrame({'Column': missing_values.index, 'Missing Values': missing_values.values})
missing_data['Percentage'] = (missing_data['Missing Values'] / len(data)) * 100
print("Missing Values in Each Column:")
print(missing_data)

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='Column', y='Missing Values', data=missing_data, palette='viridis')
plt.title('Missing Values per Column', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.xlabel('Columns', fontsize=12)
plt.ylabel('Number of Missing Values', fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

Thus, there are no missing values

# Feature Engineering

In [ ]:
def basic_feature_engineering(df):
    # Create copy to avoid modifying original data
    df_processed = df.copy()

    # Timestamp processing
    if 'DateTime' in df_processed.columns:
        df_processed['timestamp'] = pd.to_datetime(df_processed['timestamp'])
        df_processed['hour'] = df_processed['timestamp'].dt.hour
        df_processed['day_of_week'] = df_processed['timestamp'].dt.dayofweek

    # Transaction amount features
    if 'Quantity' in df_processed.columns:  # Check for correct column name: 'Quantity'
        df_processed['quantity_log'] = np.log1p(df_processed['Quantity'])

    # Scale numerical features
    numeric_columns = df_processed.select_dtypes(include=['float64', 'int64']).columns
    scaler = StandardScaler()
    df_processed[numeric_columns] = scaler.fit_transform(df_processed[numeric_columns])

    return df_processed

data = basic_feature_engineering(data)

In [ ]:
# Convert 'DateTime (UTC)' to datetime format if it's not already
data['DateTime (UTC)'] = pd.to_datetime(data['DateTime (UTC)'])

# Extract year, month, and day as new features
data['Year'] = data['DateTime (UTC)'].dt.year
data['Month'] = data['DateTime (UTC)'].dt.month
data['Day'] = data['DateTime (UTC)'].dt.day

data.head()


In [ ]:
# Create bins for Quantity
bins = [0, 100, 500, 1000, 5000, np.inf]
labels = ['0-100', '101-500', '501-1000', '1001-5000', '5000+']
data['Quantity_Binned'] = pd.cut(data['Quantity'], bins=bins, labels=labels)

# Display the updated dataframe with binned quantities
data.tail()


In [ ]:
# Sort by DateTime to ensure proper difference calculation
data.sort_values(by='DateTime (UTC)', inplace=True)

# Calculate time difference between transactions in seconds
data['Time_Diff'] = data['DateTime (UTC)'].diff().dt.total_seconds()

data.head()


In [ ]:
# Fill missing values with forward fill method
data.fillna(method='ffill', inplace=True)

# data.dropna(inplace=True)

In [ ]:
# Create lagged features for Quantity
for lag in range(1, 4):
    data[f'Quantity_Lag_{lag}'] = data['Quantity'].shift(lag)

data.head()


In [ ]:
# Calculate rolling mean and standard deviation for Quantity over a window of 5 periods
data['Quantity_Rolling_Mean'] = data['Quantity'].rolling(window=5).mean()
data['Quantity_Rolling_Std'] = data['Quantity'].rolling(window=5).std()

data.head()


In [ ]:
# Identify self-trades
data['Is_Self_Trade'] = data['From'] == data['To']

# Count the number of self-trades per account
self_trade_counts = data[data['Is_Self_Trade']].groupby('From').size().reset_index(name='Self_Trade_Count')

# Merge self-trade counts back to the original dataframe
data = data.merge(self_trade_counts, how='left', left_on='From', right_on='From')
data['Self_Trade_Count'].fillna(0, inplace=True)

data.head()


In [ ]:
# Create a function to detect circular trades
def detect_circular_trades(df):
    circular_trades = []
    grouped = df.groupby(['From', 'To'])

    for (from_address, to_address), group in grouped:
        if len(group) > 1:
            circular_trades.append((from_address, to_address))

    return circular_trades

# Apply the circular trade detection function
circular_trade_pairs = detect_circular_trades(data)

# Mark circular trades in the dataframe
for from_addr, to_addr in circular_trade_pairs:
    data.loc[(data['From'] == from_addr) & (data['To'] == to_addr), 'Is_Circular_Trade'] = True

data['Is_Circular_Trade'].fillna(False, inplace=True)

data.head()


In [ ]:
# Calculate trade frequency per account
trade_frequency = data.groupby('From')['Transaction Hash'].count().reset_index(name='Trade_Frequency')

# Merge trade frequency back to the original dataframe
data = data.merge(trade_frequency, how='left', left_on='From', right_on='From')

data.head()


In [ ]:
# Create a function to identify common funders
def identify_common_funders(df):
    funder_map = {}

    for _, row in df.iterrows():
        if row['Quantity'] > 0:  # Only consider positive quantities for funding transactions
            funder_map.setdefault(row['To'], []).append(row['From'])

    return funder_map

common_funders = identify_common_funders(data)

# Convert common funders dictionary to DataFrame for analysis
funder_df = pd.DataFrame([(k, v) for k, v in common_funders.items()], columns=['Account', 'Funded_By'])

funder_df.head()


In [ ]:

data['DateTime'] = pd.to_datetime(data['DateTime (UTC)'])

# Set time window
time_window = '1H'

# Calculate transaction frequency
transaction_frequency = data.groupby(['From', pd.Grouper(key='DateTime', freq=time_window)]).size().reset_index(name='Transaction_Count')

# Merge back to original DataFrame
data = data.merge(transaction_frequency, on=['From', 'DateTime'], how='left')
data.head()


In [ ]:
# Calculate average price per token over a rolling window
data['Average_Price'] = data['Quantity'] / data['Quantity'].rolling(window=10).mean()

# Identify significant price deviations
data['Price_Manipulation_Ratio'] = data['Quantity'] / data['Average_Price']


In [ ]:
# Sort by DateTime to ensure correct order
data.sort_values(by=['From', 'DateTime'], inplace=True)

# Calculate time difference in seconds
data['Time_Diff'] = data.groupby('From')['DateTime'].diff().dt.total_seconds()
data.head()

In [ ]:
# Calculate rolling standard deviation of quantity traded
data['Volume_Std_Dev'] = data.groupby('From')['Quantity'].transform(lambda x: x.rolling(window=10).std())

# Calculate consistency score
data['Volume_Consistency_Score'] = 1 / (1 + data['Volume_Std_Dev'])
data.head()

In [ ]:
# Group by pairs and calculate transfer ratios
transfer_counts = data.groupby(['From', 'To']).size().reset_index(name='Transfer_Count')
total_transfers = data.groupby('From').size().reset_index(name='Total_Transfers')

# Merge and calculate transfer ratio
transfer_counts = transfer_counts.merge(total_transfers, on='From')
transfer_counts['Transfer_Ratio'] = transfer_counts['Transfer_Count'] / transfer_counts['Total_Transfers']
data.head()

In [ ]:
# Calculate cumulative trade volume for each address
data['Cumulative_Volume'] = data.groupby('From')['Quantity'].cumsum()
data.tail()

In [ ]:
# Count transactions between pairs of addresses
pair_counts = data.groupby(['From', 'To']).size().reset_index(name='Pair_Transaction_Count')
data = data.merge(pair_counts, on=['From', 'To'], how='left')
data.tail()

In [ ]:
# Calculate holding time for each transaction
data['Holding_Time'] = data.groupby('From')['DateTime'].diff().dt.total_seconds()

# Calculate average holding time per address
average_holding_time = data.groupby('From')['Holding_Time'].mean().reset_index(name='Average_Holding_Time')
data = data.merge(average_holding_time, on='From', how='left')
data.tail()

In [ ]:
# Calculate average transaction size per address
avg_transaction_size = data.groupby('From')['Quantity'].mean().reset_index(name='Avg_Transaction_Size')

# Merge and calculate discrepancy
data = data.merge(avg_transaction_size, on='From', how='left')
data['Amount_Discrepancy'] = (data['Quantity'] - data['Avg_Transaction_Size']).abs()
data.tail()

In [ ]:
# Calculate total volume and transaction count per address
volume_count = data.groupby('From').agg({'Quantity': 'sum', 'Transaction Hash': 'count'}).reset_index()
volume_count.rename(columns={'Quantity': 'Total_Volume', 'Transaction Hash': 'Transaction_Count'}, inplace=True)

# Calculate volume-to-transaction ratio
volume_count['Volume_to_Transaction_Ratio'] = volume_count['Total_Volume'] / volume_count['Transaction_Count']
data = data.merge(volume_count[['From', 'Volume_to_Transaction_Ratio']], on='From', how='left')
data.tail()

In [ ]:
# Count unique tokens traded by each address
token_diversity = data.groupby('From')['To'].nunique().reset_index(name='Token_Diversity_Index')
df = data.merge(token_diversity, on='From', how='left')
data.tail()

In [ ]:
# Set DateTime as index and resample to count daily transactions
daily_transaction_count = data.set_index('DateTime (UTC)').groupby('From').resample('D').size().reset_index(name='Daily_Transaction_Count')

# Merge back to original DataFrame
data = data.merge(daily_transaction_count, on=['From', 'DateTime (UTC)'], how='left')
data.tail()

In [ ]:
# Calculate total transfers for each address pair and overall transfers for each address
total_transfers = data.groupby('From').size().reset_index(name='Total_Transfers')
pair_transfer_counts = data.groupby(['From', 'To']).size().reset_index(name='Pair_Transfer_Count')

# Merge and calculate transfer ratio
pair_transfer_counts = pair_transfer_counts.merge(total_transfers, on='From')
pair_transfer_counts['Transfer_Ratio'] = pair_transfer_counts['Pair_Transfer_Count'] / pair_transfer_counts['Total_Transfers']

# Merge back to original DataFrame if needed
data = data.merge(pair_transfer_counts[['From', 'To', 'Transfer_Ratio']], on=['From', 'To'], how='left')
data.tail()

In [ ]:
plt.scatter(range(sample_data.shape[0]), np.sort(sample_data['Quantity'].values))
plt.xlabel('Frequency')
plt.ylabel('Quantity')
plt.title("Quantity Distribution")
sns.despine()
plt.show()

In [ ]:
sns.distplot(sample_data['Quantity'])
plt.title("Quantity Distribution")
sns.despine()
plt.show()

In [ ]:
print("Skewness: %f" % sample_data['Quantity'].skew())
print("Kurtosis: %f" % sample_data['Quantity'].kurt())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assuming 'data' is your DataFrame
features = ['Quantity', 'UnixTimestamp', 'Blockno', 'Time_Diff', 'Quantity_Rolling_Mean', 'Quantity_Rolling_Std', 'Total_Traded_Volume', 'Trade_Frequency']

X = data[features]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data (80% training, 20% testing)
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

